## Driver code comparing output of different preproc pipelines 
- Note: currently using output after atlas-based grouping
- Atlas used: aparc (Freesurfer) DKT-31 Mindboggle (ANTs: https://mindboggle.readthedocs.io/en/latest/labels.html) 

### Steps
- import data csvs 
- visualize data distributions 
- correlate features across pipelines
- compare performance of machine-learning model (scikit-learn)
- compare performance of statsmodels (ols or logit)

In [1]:
import sys
import numpy as np
import pandas as pd
import itertools

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

sys.path.append('../')
from lib.data_handling import *
from lib.data_stats import *

### Data paths

In [2]:
#proj_dir = '/Users/nikhil/code/git_repos/compare-surf-tools/'
proj_dir = '/Users/nikhil/projects/compare-surf-tools/'
data_dir = proj_dir + 'data/'
fs60_dir = data_dir + 'fs60_group_stats/'
results_dir = data_dir + 'results/'

demograph_file = 'ABIDE_Phenotype.csv'
dkt_roi_names = 'DKT_parcel_map_FS_CIVET.csv'

ants_file = 'ABIDE_ants_thickness_data.csv' #uses modified (mindboggle) dkt atlas with 31 ROIs
civet_file = 'ABIDE_civet2.1_thickness.csv'

fs53_file = 'ABIDE_fs5.3_thickness.csv'
fs51_file = 'cortical_fs5.1_measuresenigma_thickavg.csv' 
fs60_lh_file = 'lh.aparc.thickness.table.test1' #'aparc_lh_thickness_table.txt' #'lh.aparc.thickness.table.test1'
fs60_rh_file = 'rh.aparc.thickness.table.test1' #'aparc_rh_thickness_table.txt' #'rh.aparc.thickness.table.test1'


### Global Vars

In [3]:
subject_ID_col = 'SubjID'

### Load data

In [4]:
# Demographics and Dx
demograph = pd.read_csv(data_dir + demograph_file)
demograph = demograph.rename(columns={'Subject_ID':subject_ID_col})

# ROI names
dkt_roi_map = pd.read_csv(data_dir + dkt_roi_names)

# CIVET 2.1
civet_data = pd.read_csv(data_dir + civet_file)
print('shape of civet data {}'.format(civet_data.shape))
civet_data_std = standardize_civet_data(civet_data, subject_ID_col, dkt_roi_map)
print('shape of stdized civet data {}'.format(civet_data_std.shape))
print('')

# ANTs
ants_data = pd.read_csv(data_dir + ants_file, header=2)
print('shape of ants data {}'.format(ants_data.shape))
ants_data_std = standardize_ants_data(ants_data, subject_ID_col)
print('shape of stdized ants data {}'.format(ants_data_std.shape))
print('')

# FS
fs53_data = pd.read_csv(data_dir + fs53_file)
print('shape of fs53 data {}'.format(fs53_data.shape))
fs53_data_std = standardize_fs_data(fs53_data, subject_ID_col)
print('shape of stdized fs53 data {}'.format(fs53_data_std.shape))
print('')

fs51_data = pd.read_csv(data_dir + fs51_file)
print('shape of fs51 data {}'.format(fs51_data.shape))
fs51_data_std = standardize_fs_data(fs51_data, subject_ID_col)
print('shape of stdized fs51 data {}'.format(fs51_data_std.shape))
print('')

fs60_lh_data = pd.read_csv(fs60_dir + fs60_lh_file, delim_whitespace=True)
fs60_rh_data = pd.read_csv(fs60_dir + fs60_rh_file, delim_whitespace=True)
print('shape of fs60 data l: {}, r: {}'.format(fs60_lh_data.shape,fs60_rh_data.shape))

fs60_data_std = standardize_fs60_data(fs60_lh_data, fs60_rh_data, subject_ID_col)
print('shape of stdized fs51 data {}'.format(fs60_data_std.shape))

shape of civet data (3, 65)
shape of stdized civet data (3, 65)

shape of ants data (1101, 99)
shape of stdized ants data (1101, 90)

shape of fs53 data (976, 74)
shape of stdized fs53 data (976, 74)

shape of fs51 data (1112, 74)
shape of stdized fs51 data (1112, 74)

shape of fs60 data l: (1047, 36), r: (1047, 36)
shape of left and right merge fs6.0 df (1047, 71)
shape of stdized fs51 data (1047, 71)


### Create master dataframe

In [5]:
data_dict = {'civet': civet_data_std,
            'ants' : ants_data_std,
            'fs60' : fs60_data_std,
            'fs53' : fs53_data_std,
            'fs51' : fs51_data_std}

na_action = 'drop' # options: ignore, drop; anything else will not use the dataframe for analysis. 
master_df, common_subs, common_roi_cols = combine_processed_data(data_dict, subject_ID_col, na_action)

print('\nNumber of common ROIs {}'.format(len(common_roi_cols)))

# Add demographic columns to the master_df
useful_demograph = demograph[[subject_ID_col,'SEX','AGE_AT_SCAN','DX_GROUP','SITE_ID']].copy()

# Shift to (0 and 1 instead of 1 and 2 for statsmodels)
useful_demograph['DX_GROUP'] = useful_demograph['DX_GROUP']-1
useful_demograph['SEX'] = useful_demograph['SEX']-1
_,useful_demograph[subject_ID_col] = useful_demograph[subject_ID_col].str.rsplit('_', 1).str

master_df = pd.merge(master_df, useful_demograph, how='left', on=subject_ID_col)
print('\nmaster df shape after adding demographic info {}'.format(master_df.shape))

Number of datasets: 5
Finding common subject and columns
dataset : civet
common subs: 3
dataset : ants
common subs: 0
dataset : fs60
common subs: 1047
dataset : fs53
common subs: 942
dataset : fs51
common subs: 942
Number of common subjects and columns: 942, 63

checking civet dataframe
Shape of the dataframe based on common cols and subs (3, 63)
Basic data check passed
Shape of the concat dataframe (3, 64)

checking ants dataframe
Shape of the dataframe based on common cols and subs (941, 63)
Basic data check passed
Shape of the concat dataframe (944, 64)

checking fs60 dataframe
Shape of the dataframe based on common cols and subs (942, 63)
Basic data check passed
Shape of the concat dataframe (1886, 64)

checking fs53 dataframe
Shape of the dataframe based on common cols and subs (942, 63)
Basic data check passed
Shape of the concat dataframe (2828, 64)

checking fs51 dataframe
Shape of the dataframe based on common cols and subs (942, 63)
Basic data check passed
Shape of the concat

### Correlation between pipelines

In [6]:
possible_pairs = list(itertools.combinations(data_dict.keys(), 2))

for pair in possible_pairs:
    pipe1 = pair[0]
    pipe2 = pair[1]
    df1 = master_df[master_df['pipeline']==pipe1][[subject_ID_col]+common_roi_cols]
    df2 = master_df[master_df['pipeline']==pipe2][[subject_ID_col]+common_roi_cols]

    xcorr = cross_correlations(df1,df2,subject_ID_col)
    print('Avg cross correlation between {} & {} = {:4.2f}\n'.format(pipe1,pipe2,np.mean(xcorr)))

Avg cross correlation between fs51 & ants = 0.43

Avg cross correlation between fs51 & fs53 = 0.89

Avg cross correlation between fs51 & fs60 = 0.86

Avg cross correlation between ants & fs53 = 0.47

Avg cross correlation between ants & fs60 = 0.43

Avg cross correlation between fs53 & fs60 = 0.91



### Compare ML performance 

In [7]:
roi_cols = common_roi_cols
covar_continuous_cols = [] #'AGE_AT_SCAN'
covar_cat_cols = [] #'SEX','SITE_ID','DX_GROUP'

model_type = 'classification'

if model_type.lower() == 'regression':
    outcome_col = 'AGE_AT_SCAN'
    model = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
else: 
    outcome_col = 'DX_GROUP'
    model = svm.SVC(kernel='linear')
    #model = RandomForestClassifier(n_estimators=50, max_depth=2,random_state=0)

ml_perf = computePipelineMLModels(master_df,roi_cols,covar_continuous_cols,covar_cat_cols,outcome_col,
                                  model_type,model)

Running ML classifer on 4 pipelines
Pipeline fs60
Data shapes X (941, 62), y 941 ([506, 435])
Using classification model with perf metric roc_auc
 Perf mean:0.566, sd:0.059
Pipeline ants
Data shapes X (941, 62), y 941 ([506, 435])
Using classification model with perf metric roc_auc
 Perf mean:0.577, sd:0.046
Pipeline fs51
Data shapes X (941, 62), y 941 ([506, 435])
Using classification model with perf metric roc_auc
 Perf mean:0.545, sd:0.059
Pipeline fs53
Data shapes X (941, 62), y 941 ([506, 435])
Using classification model with perf metric roc_auc
 Perf mean:0.545, sd:0.053



### Compare statsmodels performance 

In [7]:
save_sm_perf = False
roi_cols = common_roi_cols
covar_continuous_cols = ['AGE_AT_SCAN']
covar_cat_cols = ['SEX','SITE_ID']
outcome_col = 'DX_GROUP' #AGE_AT_SCAN #DX_GROUP #SEX
stat_model = 'logit' #ols #logit

sm_perf = computePipelineStatsModels(master_df,roi_cols,covar_continuous_cols,covar_cat_cols,
                                     outcome_col,stat_model)
print('Shape of the stats_models results df {}'.format(sm_perf.shape))

if save_sm_perf:
    save_path = '{}pipelines_sm_perf_{}.pkl'.format(results_dir,outcome_col)
    print('Saving sm_perf dictionary at \n{}'.format(save_path))
    sm_perf.to_pickle(save_path)


Running 62 mass-univariate logit statsmodels on 4 pipelines
Pipeline fs51
Example statsmodel run:
 DX_GROUP ~ R_precentral + AGE_AT_SCAN + C(SEX) + C(SITE_ID)
Top 10 significant regions:
                        roi     t_val     p_val pipeline
12               L_lingual -3.198658  0.001381     fs51
8   L_rostralmiddlefrontal -2.978978  0.002892     fs51
48               R_lingual -2.968617  0.002991     fs51
54      L_lateraloccipital -2.962219  0.003054     fs51
17                L_cuneus -2.919661  0.003504     fs51
50    L_posteriorcingulate -2.914745  0.003560     fs51
1       R_lateraloccipital -2.862227  0.004207     fs51
33         L_pericalcarine -2.675520  0.007461     fs51
58           R_postcentral -2.653052  0.007977     fs51
55  R_rostralmiddlefrontal -2.580449  0.009867     fs51
Pipeline ants
Example statsmodel run:
 DX_GROUP ~ R_precentral + AGE_AT_SCAN + C(SEX) + C(SITE_ID)
Top 10 significant regions:
                        roi     t_val     p_val pipeline
41          